In [4]:
import os
import pandas as pd

data_photos = pd.DataFrame(pd.read_csv("pattern_info.csv")).dropna()

In [5]:
data_photos.columns

Index(['Unnamed: 0', 'name', 'permalink', 'square_url', 'small_url'], dtype='object')

In [6]:
import requests
from io import BytesIO
from PIL import Image
import torch
from torchvision import transforms

#images = data_photos["photos_square_url"]
#image_url = images.iloc[0]
##response = requests.get(image_url)
#img = Image.open(BytesIO(response.content))
#convert_tensor = transforms.ToTensor()
#img_tensor = convert_tensor(img)


In [7]:
import warnings 
warnings.filterwarnings('ignore')

import torch
from torchvision import transforms
import torchvision

from sklearn.preprocessing import LabelEncoder

import random

import requests
from io import BytesIO
import matplotlib.pyplot as plt

encoder = LabelEncoder()

## splitting the data for ease
labels = data_photos[['name', 'permalink']]
labels["type_encoded"] =  encoder.fit_transform(labels['permalink'])
images = data_photos["small_url"]

## Running counters
fail_counter = 0
success_counter = 0
clean_index = []

## Hyperparameters?
n_values = 5000
possible_values = range(0,data_photos.shape[0])
random_set = random.sample(possible_values,n_values)
print(random_set[0:100])

for i in random_set:
    ## Try needed as some links are broken or go to no longer existing photos
    try:
        ## Loading in Image through URL
        image_url = images.iloc[i]
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content))
        ## Tensoring it
        convert_tensor = transforms.ToTensor()
        img_tensor = convert_tensor(img)
        ## Resizing and joining it to others
        # they all need to be the same size so I'm just sayin 250x250 lmao
        #resize=torchvision.transforms.Resize((250,250))
        # img_resize = resize(img_tensor)
        ## First round of making the clean images a tensor
        if success_counter == 0:
            clean_images = img_tensor
            success_counter = success_counter + 1
        ## second round of stacking the tensors to add another Batch dimension
        elif success_counter == 1:
            clean_images = torch.stack([clean_images,img_tensor])
            success_counter = success_counter + 1
            print("check")
        ## For the rest of the rounds concatinating the code
        else:
            img_tensor = img_tensor.unsqueeze(dim = 3).permute(3,0,1, 2)
            clean_images = torch.concat([clean_images,img_tensor])
        ## Making sure Labels are up to date
        clean_index.append(i)
    except:
        ## To see how many we lost
        fail_counter += 1

clean_labels = labels.iloc[clean_index,:]
print("Zoro,'Nothing happened' x " + str(fail_counter))
print(clean_labels.shape)
print(clean_images.shape)

[594, 1938, 5193, 1194, 4352, 1918, 42, 593, 3043, 3085, 65, 1306, 5646, 1454, 5000, 1632, 1474, 4540, 3622, 2285, 5206, 3006, 2836, 2329, 3409, 6047, 736, 741, 3757, 2432, 1751, 4610, 580, 4747, 6096, 4857, 6031, 2968, 2095, 4894, 5024, 2466, 2746, 3367, 1072, 3424, 4133, 2886, 1092, 5349, 4963, 2572, 615, 3246, 3917, 1561, 5245, 4608, 2139, 1954, 6468, 5822, 3984, 4245, 1557, 6217, 1464, 312, 5386, 5412, 461, 5047, 4401, 1941, 5814, 3345, 5379, 355, 2826, 1313, 148, 3116, 5005, 872, 2266, 2470, 3530, 288, 219, 1298, 3670, 4193, 5447, 4218, 2193, 5483, 3279, 310, 32, 6244]
check
Zoro,'Nothing happened' x 3951
(1049, 3)
torch.Size([1049, 3, 240, 180])


In [8]:
clean_labels.iloc[:,1]

594     cardigan
4352    camisole
1454         tee
736     pullover
741     pullover
          ...   
747     pullover
6077    pullover
5451         tee
5699       scarf
3715    camisole
Name: permalink, Length: 1049, dtype: object

In [9]:
clean_label = pd.get_dummies(clean_labels, columns = ["permalink"], prefix = "type", dtype = float)
clean_label = clean_label.iloc[:,2:]
clean_label.shape

(1049, 18)

In [10]:
import torch
import numpy as np

num_final = clean_labels.shape[0]
split_val = .8


train_images = clean_images[1:round(split_val*num_final),...].float()
test_images = clean_images[round(split_val*num_final):num_final,...].float()

train_labels = torch.from_numpy(clean_label.iloc[1:round(split_val*num_final),:].values).float()
test_labels = torch.from_numpy(clean_label.iloc[round(split_val*num_final):num_final,:].values).float()

mean = np.mean(clean_labels.iloc[:,2].values)
std = np.std(clean_labels.iloc[:,2].values)
#train_labels = (train_labels - mean) / std
#test_labels = (test_labels - mean) / std


print(train_images.shape)
print(test_images.shape)

print(train_labels.shape)
print(test_labels.shape)

torch.Size([838, 3, 240, 180])
torch.Size([210, 3, 240, 180])
torch.Size([838, 18])
torch.Size([210, 18])


In [11]:
train_labels

tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.]])

In [12]:
train_images.type()

'torch.FloatTensor'

In [13]:
import torch.utils.data as data

batch_size_mod = 10
batch_size = round(train_labels.shape[0]/batch_size_mod)

loader = data.DataLoader(data.TensorDataset(train_images, train_labels), batch_size=batch_size)
testloader = data.DataLoader(data.TensorDataset(test_images, test_labels), batch_size=batch_size)

In [17]:
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader


class ConvNeuralNet(nn.Module):
    def __init__(self):
        super(ConvNeuralNet, self).__init__()
        ## Xavier weight initialization
        self.layer1 = nn.Sequential(
            # adjusting input size for the 3 layered image
            nn.Conv2d(3, 6, 5),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        ## Xavier initial weights
        nn.init.xavier_normal_(self.layer1[0].weight)
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, 7),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(35200, 10000)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(.01)
        self.fc1 = nn.Linear(10000, 1000)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(1000, 20)

        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        ## Droupout layer
        out = self.dropout(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [18]:
model = ConvNeuralNet()

## Established loss function
cost = nn.CrossEntropyLoss()
## SGD as optimizer, L@ regularization brought accuracy down
optimizer = torch.optim.SGD(model.parameters(), lr=.0001,momentum=0.9)  

In [19]:
num_epochs = 15

min_loss = 1000000

total_step = len(loader)
for epoch in range(num_epochs):
    for i, data in enumerate(loader):  
        

        images, labels = data
        optimizer.zero_grad()

        #Forward pass
        outputs = model(images)
        loss = cost(outputs, labels)
            
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                        .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

        if loss.item() < min_loss:
            torch.save(model.state_dict(), 'ideal_model.pth')
            min_loss = loss.item()

RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [ ]:
model = ConvNeuralNet()
model.load_state_dict(torch.load("ideal_model.pth"))

dataiter = iter(testloader)
images, y_train = next(dataiter)
y_pred = model(images)

ce = cost(y_pred, y_train)
acc = (torch.argmax(y_pred, 1) == torch.argmax(y_train, 1)).float().mean()

print(ce)
print(acc)

tensor(2.8424, grad_fn=<DivBackward1>)
tensor(0.2861)


In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        acc = (torch.argmax(outputs, 1) == torch.argmax(labels, 1)).float().mean()

        print(f'Accuracy of the network on the 10 test images: {acc*100} %')

Accuracy of the network on the 10 test images: 28.607595443725586 %
Accuracy of the network on the 10 test images: 29.620254516601562 %
Accuracy of the network on the 10 test images: 30.30303192138672 %
